$ real\_value = scale * ( quantized\_value - zero\_point ) $

$ quantized\_value = int(round(\frac{real\_value}{scale} + zero\_point) $

To calculate the scale and zero poinnt values, the following can be used. 


In [84]:
import torch

input = torch.rand((3,3), dtype=torch.float32)*250

rmax = input.max().item()
rmin = input.min().item()
qtype = torch.iinfo(torch.int8)

scale = (rmax - rmin)/(qtype.max - qtype.min)
zero_point = round(qtype.min - rmin/scale)

print ("scale, zero_point = ", scale, zero_point)
print(input)

def quantize_tensor(scale, real_value, zero_point, target_dtype=torch.int8):
    scaled_and_shifted_tensor = real_value / scale + zero_point

    rounded_tensor = torch.round(scaled_and_shifted_tensor)

    q_min = torch.iinfo(target_dtype).min
    q_max = torch.iinfo(target_dtype).max

    q_tensor = rounded_tensor.clamp(q_min,q_max).to(target_dtype)
    
    return q_tensor


qt = quantize_tensor(scale, input, zero_point)
print(qt)
def dequantize_tensor(scale, quantized_value, zero_point, target_dtype=torch.float32):
    return (scale*(quantized_value.float()-zero_point))

dqt = dequantize_tensor(scale, qt, zero_point)
print(dqt)

print ("RMS error = ", (dqt - input).square().mean())

scale, zero_point =  0.7576206955255247 -152
tensor([[156.1777,  37.9028,  86.1635],
        [211.0878,  28.3854, 190.6095],
        [118.6020,  17.8945,  84.2996]])
tensor([[  54, -102,  -38],
        [ 127, -115,  100],
        [   5, -128,  -41]], dtype=torch.int8)
tensor([[156.0699,  37.8810,  86.3688],
        [211.3762,  28.0320, 190.9204],
        [118.9464,  18.1829,  84.0959]])
RMS error =  tensor(0.0669)
